In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [4]:
df = pd.read_csv("assets.csv")
df = df[df["type"] == "model"]
#df.drop(columns=["sample", "datasheet", "included", "excluded", "adaptation", "output_space", "terms_of_service", "monthly_active_users", "user_distribution", "failures"], inplace=True)
df["created_date"] = pd.to_datetime(df["created_date"], format="%d/%m/%Y")
df["inputs"] = df.modality.str.split("; ").str[0].str.split(", ")
df["outputs"] = df.modality.str.split("; ").str[1].str.split(", ")
df.drop(columns="modality", inplace = True)
df["size"] = df["size"].apply(lambda x: np.nan if x == "unknown" else x)
df["dense"] = df["size"].str.endswith("(dense)")
df["dense"].fillna(False, inplace=True)
df["parameters"] = df["size"].str.extract(r"^(\d+[a-zA-Z])")
df["parameters_(millions)"] = df["parameters"].str.replace("M", "").str.replace("B", "000").str.replace("T", "000000")
df["parameters_(millions)"] = pd.to_numeric(df["parameters_(millions)"], downcast="integer")
df.drop(columns=["size", "parameters"], inplace=True)
df["dependencies"] = df["dependencies"].apply(lambda x: x[1:-1].split(','))
df["analysis"].fillna("unknown", inplace=True)
df["access"] = df["access"].map({"open":2, "limited":1, "closed": 0})
df["training_time"].fillna("unknown", inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 359 entries, 3 to 564
Data columns (total 33 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   type                   359 non-null    object        
 1   name                   359 non-null    object        
 2   organization           359 non-null    object        
 3   description            299 non-null    object        
 4   created_date           357 non-null    datetime64[ns]
 5   url                    357 non-null    object        
 6   datasheet              0 non-null      object        
 7   sample                 0 non-null      object        
 8   analysis               359 non-null    object        
 9   dependencies           359 non-null    object        
 10  included               0 non-null      object        
 11  excluded               0 non-null      object        
 12  quality_control        85 non-null     object        
 13  access    

In [28]:
pd.options.display.max_columns = None

In [36]:
df[df["name"].str.contains("Mistral")]

,type,name,organization,description,created_date,url,datasheet,sample,analysis,dependencies,included,excluded,quality_control,access,license,intended_uses,prohibited_uses,monitoring,feedback,model_card,training_emissions,training_time,training_hardware,adaptation,output_space,terms_of_service,monthly_active_users,user_distribution,failures,inputs,outputs,dense,parameters_(millions)
34,model,BioMistral,"Avignon University, Nantes University",BioMistral is an open-source Large Language Mo...,2024-02-15,https://arxiv.org/pdf/2402.10373.pdf,NaN,NaN,BioMistral was evaluated on a benchmark compri...,"['Mistral', 'PubMed Central']",NaN,NaN,NaN,2,Apache 2.0,"Research in the biomedical domain, especially ...",Prohibited from deploying in production enviro...,NaN,https://huggingface.co/BioMistral/BioMistral-7...,https://huggingface.co/BioMistral/BioMistral-7B,unknown,unknown,32 NVIDIA A100 80GB GPUs,NaN,NaN,NaN,NaN,NaN,NaN,[text],[text],True,7000.0
64,model,Mistral,Mistral AI,Mistral is a compact language model.,2023-09-27,https://mistral.ai/news/announcing-mistral-7b/,NaN,NaN,Evaluated in comparison to LLaMA series models...,[],NaN,NaN,NaN,2,Apache 2.0,NaN,NaN,NaN,https://huggingface.co/mistralai/Mistral-7B-v0...,https://huggingface.co/mistralai/Mistral-7B-v0.1,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,[text],[text],True,NaN
65,model,Mistral Large,Mistral AI,Mistral Large is Mistral AI’s new cutting-edge...,2024-02-26,https://mistral.ai/news/mistral-large/,NaN,NaN,Evaluated on commonly used benchmarks in compa...,[],NaN,NaN,NaN,1,unknown,NaN,NaN,NaN,NaN,NaN,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,[text],[text],False,NaN
520,model,YaRN Mistral,"Nous Research, EleutherAI, University of Geneva",YaRN Mistral is an adapted version of Mistral ...,2023-11-01,https://arxiv.org/pdf/2309.00071.pdf,NaN,NaN,Evaluated across a variety of standard benchma...,['Mistral'],NaN,NaN,NaN,2,MIT,NaN,NaN,unknown,https://huggingface.co/NousResearch/Yarn-Mistr...,https://huggingface.co/NousResearch/Yarn-Mistr...,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,[text],[text],True,7000.0
521,model,OpenHermes 2.5 Mistral,Nous Research,OpenHermes 2.5 Mistral 7B is a state of the ar...,2023-11-03,https://huggingface.co/teknium/OpenHermes-2.5-...,NaN,NaN,Evaluated on common LLM benchmarks in comparis...,['Mistral'],NaN,NaN,NaN,2,Apache 2.0,NaN,NaN,unknown,https://huggingface.co/teknium/OpenHermes-2.5-...,https://huggingface.co/teknium/OpenHermes-2.5-...,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,[text],[text],True,7000.0
522,model,Hermes 2 Pro-Mistral,Nous,"Hermes 2 Pro on Mistral 7B is an upgraded, ret...",2024-03-10,https://huggingface.co/NousResearch/Hermes-2-P...,NaN,NaN,The model was examined across a range of bench...,"['Mistral', 'OpenHermes 2.5 Dataset', 'Nous ...",NaN,NaN,"The model was evaluated across multiple tasks,...",2,Apache 2.0,The model is intended for general task and con...,unknown,unknown,https://huggingface.co/NousResearch/Hermes-2-P...,https://huggingface.co/NousResearch/Hermes-2-P...,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,[text],[text],True,7000.0
561,model,SciPhi Mistral,SciPhi,SciPhi Mistral is a Large Language Model (LLM)...,2023-11-07,https://huggingface.co/SciPhi/SciPhi-Mistral-7...,NaN,NaN,unknown,['Mistral'],NaN,NaN,NaN,2,MIT,NaN,NaN,unknown,https://huggingface.co/SciPhi/SciPhi-Mistral-7...,https://huggingface.co/SciPhi/SciPhi-Mistral-7...,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,[text],[text],True,7000.0


In [35]:
df[df["name"] == "Vicuna"]

,type,name,organization,description,created_date,url,datasheet,sample,analysis,dependencies,included,excluded,quality_control,access,license,intended_uses,prohibited_uses,monitoring,feedback,model_card,training_emissions,training_time,training_hardware,adaptation,output_space,terms_of_service,monthly_active_users,user_distribution,failures,inputs,outputs,dense,parameters_(millions)
428,model,Vicuna,LMSYS,An open-source chatbot trained by fine-tuning ...,2023-03-30,https://lmsys.org/blog/2023-03-30-vicuna/,NaN,NaN,Evaluated against similar LLMs using GPT-4 as ...,"['LLaMA', 'ShareGPT conversations data']",NaN,NaN,NaN,2,Apache 2.0,research on LLMs and chatbots,NaN,NaN,https://huggingface.co/datasets/bigcode/the-st...,https://huggingface.co/lmsys/vicuna-13b-delta-v0,NaN,1 day,8 A100 GPUs,NaN,NaN,NaN,NaN,NaN,NaN,[text],[text],True,13000.0
